In [ ]:
from landlab.components import FlowAccumulator
from       IPython.display import clear_output
from         landlab    import RasterModelGrid
import                 matplotlib.pylab as plt
from                     scipy import optimize
from                           mpmath import *
import                             numpy as np
import                                   scipy

In [ ]:
def DinfEroder(mg, dt, K_sp, m_sp, n_sp):
    # This function computes the erosion term implicitly using the D_infinity flow direction method
    '''
    Function arguments:
    mg  :                    Raster grid used for the simulation
    dt  :                                  Time-step to be taken
    K_sp:                                    Erosion coefficient
    m_sp: Exponent of specific drainage area in the erosion term
    n_sp:            Exponent of local slope in the erosion term
    '''
    
    # Initializing local arrays for the function block
    nodes      =                                                      np.arange(mg.number_of_nodes)
    receiver   =                                                  mg.at_node['flow__receiver_node']
    A          =                                               np.copy(mg.at_node['drainage_area'])
    proportion =                                           mg.at_node['flow__receiver_proportions']
    ele        =                                      np.copy(mg.at_node['topographic__elevation'])
    neighbor   = np.append(mg.diagonal_adjacent_nodes_at_node, mg.adjacent_nodes_at_node, axis = 1)
    donor      =                                                            np.zeros_like(neighbor)
    
    # Listing the receivers of each node's neighbors
    receiver_of_neighbor        =                                         receiver[neighbor]
    receiver_of_neighbor[:,:,0] = np.where(neighbor == -1 , -1, receiver_of_neighbor[:,:,0])
    receiver_of_neighbor[:,:,1] = np.where(neighbor == -1 , -1, receiver_of_neighbor[:,:,1])
    
    # Determining the donors for each node in the domain
    for nei in range(8):
        donor[:, nei] = np.where((receiver_of_neighbor[:, nei, 0] == nodes) |
                                 (receiver_of_neighbor[:, nei, 1] == nodes) , neighbor[:, nei], -1)

    # Initializing the queue by adding sink nodes
    queue           = list(np.arange(mg.number_of_nodes)[mg.at_node['flow__sink_flag'] == 1])
    
    # Marking the sink nodes as processed
    processed_nodes        =   np.zeros_like(nodes, dtype=bool)
    processed_nodes[queue] =                               True
    
    # Traversing the remaining network
    while len(queue) > 0:
        
        # Popping the first element of the queue
        node = queue.pop(0)
        
        # Visiting its donors
        for nei in donor[node, donor[node] >= 0]:
            
            # Processing the donor if all the receivers of that donor are processesd
            if processed_nodes[nei] == False and np.all(processed_nodes[receiver[nei][receiver[nei] >= 0]]):
                processed_nodes[nei] = True
                
                # Finding the slope using the elevation of neighbors in cardinal and diagonal directions
                h_0 , h_1 = None, None
                for res, pro in zip(receiver[nei], proportion[nei]):
                    if res != -1:
                        if res // ncols == nei // ncols or res % ncols == nei % ncols:
                            h_0 = ele[res]
                        else:
                            h_1 = ele[res]
                h = ele[nei]
                width = dx
                if h_0 is not None and h_1 is not None:
                    f = lambda h_next: h_next - h + K_sp * (A[nei] / width)**m_sp *\
                                ((h_0 - h_1)**2 + (h_next - h_0)**2)**(n_sp / 2.) * dt / dx**n_sp
                    min_h = min(h_0, h_1)

                elif h_0 is not None:
                    f = lambda h_next: h_next - h + K_sp * (A[nei] / width)**m_sp *\
                                ((h_next - h_0)**2)**(n_sp / 2.) * dt / dx**n_sp
                    min_h = h_0
                elif h_1 is not None:
                    f = lambda h_next: h_next - h + K_sp * (A[nei] / width)**m_sp *\
                                (h_next - h_1)**n_sp * dt / (2**.5 * dx)**n_sp

                    min_h = h_1
                    
                # Updating the elevation implicitly using Brent’s method in Python
                ele[nei] = optimize.brenth(f, h, min_h)
                
                # Appending the processed node to the queue
                queue.append(nei)
    return ele

def implicit_diffusion(zgiven, dt, D):    
    # This function implicitly solves the diffusion term with the domain boundary nodes
    # at zero fixed elevation value. For any other boundary conditions, the diagonals of the 
    # linear (matrix) system should be modified accordingly.
    '''
    Function arguments:
    dt     :          Time-step to be taken
    zgiven :                Elevation field
    D      :         Soil creep coefficient
    '''   
    # dx is grid resolution, dt is the time step, D is the coefficient
    kd   =                       D*dt/(dx**2)
    d1m1 = np.repeat(-kd, len(mg.core_nodes))
    d1m1[ncols-3:d1m1.size:ncols-2]       = 0
    A    = scipy.sparse.diags([-kd, d1m1, 1 + 4*kd, d1m1, -kd], [-ncols+2, -1, 0, 1, ncols-2], shape=(len(mg.core_nodes), len(mg.core_nodes)))
    return scipy.sparse.linalg.lgmres(A, zgiven, atol=0.000000000001)[0]

In [ ]:
#          Initializing the coefficients, domain-size, and model parameters
K_sp  = 0.0025 #                                Fluvial erosion coefficient
D     =  0.005 #                                     Soil creep coefficient
U     =  0.001 #                                                Uplift rate
m_sp  =    0.5 # Exponent of the specific drainage area in the erosion term
n_sp  =    1.0 #            Exponent of the local slope in the erosion term
dx    =    1.0 #                  Raster grid resolution for the simulation
nrows =    500 #                                             Number of rows
ncols =    100 #                                          Number of columns

CI    = (K_sp*((ncols)*dx)**(m_sp + n_sp))/(D**n_sp * U**(1-n_sp))
print("The channelization index for this run is",       round(CI))
print("This index tells the tendency of the system to form channels. More information is in the manuscript of the code.")

# Providing the initial condition for the elevation field (z)
z     = np.load('Zinitial_500_100.npy')

In [ ]:
# Initializing the raster grid
mg    =                           RasterModelGrid((nrows,ncols), dx)
_     =               mg.add_zeros('node', 'topographic__elevation')

# Setting the boundary conditions for the numerical model
for edge in (mg.nodes_at_left_edge, mg.nodes_at_right_edge):
    mg.status_at_node[edge] = RasterModelGrid.BC_NODE_IS_FIXED_VALUE
for edge in (mg.nodes_at_bottom_edge,mg.nodes_at_top_edge):
    mg.status_at_node[edge] = RasterModelGrid.BC_NODE_IS_FIXED_VALUE

# Initializing the flow-direction algorithm for the given topography    
mg.at_node['topographic__elevation']   =     z.reshape(nrows * ncols)
fc = FlowAccumulator(mg, flow_director =          'FlowDirectorDINF')
fc.run_one_step()

In [ ]:
# Lists, variables to keep track of time-steps, temporal changes in elevation, etc., during the simulation run.
count        =        0
count_max    =      500
min_try      =      500
i            =       -1
t            =        0
diff_list    =       []
steady_state =    False

# Beginning of the simulation run
while steady_state is False:
    i    += 1
    alpha = min((i + 1) * 0.01, 1.)
    dt    = alpha * min(dx / (K_sp * np.max((mg.at_node['drainage_area'] / dx) ** m_sp)) , 20. * dx ** 2 / (2 * D))
    dt    = min(dt, 500.)

    ele_1      = np.copy(mg.at_node['topographic__elevation'])
    erode_done =                                         False
    
    while erode_done is False:
        try:
            fc = FlowAccumulator(mg, flow_director = 'FlowDirectorDINF')
            fc.run_one_step()
            
            # This function implements the implicit algorithm for updating the erosion term in Anand et al.(2020).
            mg.at_node['topographic__elevation'] = DinfEroder(mg, dt, K_sp, m_sp, n_sp)
            
            # This function updates the diffusion term implicitly for the fixed elevation boundary conditions.
            mg.at_node['topographic__elevation'][mg.core_nodes] = implicit_diffusion(mg.at_node['topographic__elevation'][mg.core_nodes] + U * dt, dt, D)
            erode_done = True

        except ValueError:
            print('Adjusting dt')
            mg.at_node['topographic__elevation'] = ele_1
            dt = dt / 2.

    ele_2             = np.copy(mg.at_node['topographic__elevation'])
    t                          +=                                  dt
    ele_diff                    =         np.abs(ele_1 - ele_2).max()
    max_diff_location           =      np.abs(ele_1 - ele_2).argmax()
    ele_diff_mean               = np.abs(ele_1.mean() - ele_2.mean())
    diff_list.append([t, ele_diff, ele_diff_mean, max_diff_location])

    if ele_diff_mean < 0.00005 * U * dt:
        count = count + 1
    else:
        count = 0

    if  i >= min_try and ele_diff < U * 0.001 * dt and ele_diff_mean < 0.0001 * U * dt:
        steady_state = True

    if count == count_max:
        steady_state = True
    
    # Visualizing the transient elevation and specific drainage area solutions. If one is only
    # interested in steady-state solutions, consider commenting the generated plots for speed-up.
    if i%50 == 0:
        clear_output(wait=True)
        plt.figure(figsize=(8, 4.5))
        plt.subplot(211)
        plt.title('Specific Drainage Area [m]')
        plt.imshow(np.transpose((mg.at_node['drainage_area'].reshape(nrows,ncols)/dx)**0.5), cmap ='seismic')
        plt.colorbar()
        plt.xlabel('X [m]')
        plt.ylabel('Y [m]')
        plt.subplot(212)
        plt.title('Elevation [m]')
        plt.imshow(    np.transpose(mg.at_node['topographic__elevation'].reshape(nrows,ncols)), cmap ='BrBG')
        plt.xlabel('X [m]')
        plt.ylabel('Y [m]')
        plt.colorbar()
        plt.tight_layout()
        plt.show()

    print(i, t, round(ele_diff/ (U * 0.001 * dt), 1), round(ele_diff_mean/ (U * 0.0001 * dt), 1))

In [ ]:
#                                Saving the steady-state solution arrays
z           = str(round(chi)) + '_steady_ele_at_' + str(int(t)) + '.npy'
a           = str(round(chi)) + '_steady_acc_at_' + str(int(t)) + '.npy'
np.save(z,    mg.at_node['topographic__elevation'].reshape(nrows,ncols))
np.save(a,          mg.at_node['drainage_area'].reshape(nrows,ncols)/dx)